# Motivating work

I retrieved that notebook to document what I was thinking about when I started thinking about that...

In [147]:
import io, os, sys, json, time, string

import openai
openai.api_key="sk-XXXXX"

os.environ["LD_LIBRARY_PATH"] = "/usr/lib/x86_64-linux-gnu"
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["SERPAPI_API_KEY"] = "XXXX"

from langchain.cache import InMemoryCache
import langchain
langchain.llm_cache = InMemoryCache()

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain import OpenAI

def make_prompt_template(prompt, **kwargs):
    variables = [v[1] for v in string.Formatter().parse(prompt)]
    input_variables = []
    for var in variables:
        if var is not None and not var in kwargs:
            kwargs.update({ var : '{'+var+'}' })
            input_variables.append(var)
    return PromptTemplate(template=prompt.format(**kwargs), input_variables=input_variables)

from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio
import tqdm

base_prompt = '\n\n'.join([
    "You are the {ai_agent} automaton in the AI system {ai_name}.",
    "# Topic", "{topic}", "# {ai_name} Mind",
    "Text summary of {ai_name}' concepts and ideas on the Topic.",
    "{ai_mind}", "# Scratch Pad"
])

cold_llm = OpenAI(temperature=0.05)
hot_llm = OpenAI(temperature=0.7)

In [3]:
from serpapi import GoogleSearch

def run_search_api(query, start=0):
    start = 0
    search = GoogleSearch({
      "start": str(start), "num": "10",
      "q": query,
      "hl": "en", "gl": "us", "engine": "google_scholar",
      "google_domain": "google.com",
      "api_key": os.environ["SERPAPI_API_KEY"]
    })
    results = search.get_dict()
    results = results["organic_results"]
    results = [ [ r[k] for k in [ 'link', 'title', 'snippet' ] ] for r in results ]
    return ( results, start+len(results) )

In [4]:
query_prompt = base_prompt + """\n
You need to formulate an appropriate search query.
The query should be simple: one or two nominal groups with a single verb or conjunction.
It should be specific: it should be closely related to the "Topic being research".

Use the following format:

Reflection: identify the knowledge missing from "{ai_name} Mind" with respect to the "Topic being research"
Goal: what information do you need to obtain? 
Query: the simple and specfic query that will best fulfill your Goal

Begin!

Reflection: """
query_chain = LLMChain(llm=cold_llm, prompt=make_prompt_template(
    prompt=query_prompt,
    ai_name='Helios',
    ai_agent='search',
    input_title="Topic being researched",
    task_title="Search"
))

def run_query_chain(**kwargs):
    reflection = query_chain(kwargs)['text']
    (reflection, goal) = map(lambda x: x.strip(), reflection.split('Goal:'))
    (goal, query) = map(lambda x: x.strip(), goal.split('Query:'))
    return { 'reflection' : reflection, 'goal' : goal, 'query' : query }

In [5]:
select_prompt = base_prompt + """\n
You have reflected on the "Topic being researched", established a goal for the search, and provided a query.
You need to decide whether the item shown below is relevant for your goals.

Use the following format:

Reflection: identify the knowledge missing from "{ai_name} Mind" with respect to the "Topic being researched"
Goal: what information do you need to obtain? 
Query: the simple and specfic query that will best fulfill your Goal
Title: the title provided for the item
Identifier: a unique identifier such as a URL, file-path, physical addresse
Description: a description or snippet of the results
Observation: infer from the title, identifier, and description the information that could be found in the item
Relevant: yes/no 

Begin!

Reflection: {reflection}
Goal: {goal}
Query: {query}
Title: {item_title}
Identifier: {item_uid}
Description: {item_desc}
Observation: """
select_chain = LLMChain(llm=cold_llm, prompt=make_prompt_template(
    prompt=select_prompt,
    ai_name='Helios',
    ai_agent='search',
    input_title="Topic being researched",
    task_title="Search"
))

def run_select_chain(**kwargs):
    observation = select_chain(kwargs)['text']
    (observation,relevant) = list(map(lambda x: x.strip(), observation.split('Relevant:')))
    return { 'observation' : observation, 'relevant' : relevant }

In [16]:
summary_prompt = base_prompt + """\n
You have reflected on the "Topic being researched", established a goal for the search, and provided a query.
You have filtered the search results and selected the ones that seem relevant.
Write a summary of these finding, cross reference the item in your summary.
Use number to reference items, for example: references to the 3rd and 5th items would be "[3,5]".

Use the following format:

Reflection: identify the knowledge missing from "{ai_name} Mind" with respect to the "Topic being researched"
Goal: what information do you need to obtain? 
Query: the simple and specfic query that will best fulfill your Goal
Title 1: the title provided for the 1st item
Identifier 1: a unique identifier such as a URL, file-path, physical addresse for the 1st item
Description 1: a description or snippet of the results for the 1st item
Observation 1: your observation about this item
... Title, Identifier, Description, and Observation are repeated as many time as there are items
Categories: Identify common theme, keyword, or sources accross these items, items can be in more than one category
Category 1: Description of the category
Items 1:    list items that fit the description
... identify ut to 5 categories
Summary: Write your summary, one or two sentences per categories with citation at the end of each sentence.

Begin!

Reflection: {reflection}
Goal: {goal}
Query: {query}
{selections}
Categories: """
summary_chain = LLMChain(llm=hot_llm, prompt=make_prompt_template(
    prompt=summary_prompt,
    ai_name='Helios',
    ai_agent='search',
    input_title="Topic being researched",
    task_title="Search"
))

def run_summary_chain(**kwargs):
    categories = summary_chain(kwargs)['text']
    (categories,summary) = list(map(lambda x: x.strip(), categories.split('Summary:')))
    return { 'categories' : categories.split('\n'), 'summary' : summary }

In [17]:
def run_search(topic, ai_mind="I am a blank slate", select=3, max_pull=3):
    data = { 'topic' : topic, "ai_mind" : ai_mind }
#    print(json.dumps(data, indent=4))
    data.update(run_query_chain(**data))
#    print(json.dumps(data, indent=4))
    start = 0
    selections = []
    for i in range(max_pull):
#        print(i, start)
        (items, start) = run_search_api(data['query'], start=start)
#        print(items)
        for (item_uid, item_title, item_desc) in items:
#            print(item_uid)
            response = run_select_chain(item_uid=item_uid, item_title=item_title, item_desc=item_desc, **data)
#            print(json.dumps(response, indent=4))
            if response['relevant'].split(' ')[0].split('\n')[0].lower() == 'yes':
                selections.append({
                    'uid'   : item_uid,
                    'title' : item_title,
                    'desc'  : item_desc
                })
                selections[-1].update(**response)
        if len(selections) >= select:
            break
    selections_str = '\n'.join([
        f"""\
Title {i}: {s['title']}
Identifier {i}: {s['uid']}
Description {i}: {s['desc']}
Observation {i}: {s['observation']}"""
        for (i,s) in enumerate(selections)
    ])
    response = run_summary_chain(selections=selections_str, **data)
    data.update({ 'selections' : selections })
    data.update(response)
    return data

In [18]:
data = run_search(topic="Testing an AI agent", ai_mind="I am a blank slate", select=3, max_pull=3)
print(json.dumps(data, indent=4))

{
    "topic": "Testing an AI agent",
    "ai_mind": "I am a blank slate",
    "reflection": "Helios Mind does not have any information on testing an AI agent.",
    "goal": "To find information on how to test an AI agent.",
    "query": "How to test AI agents?",
    "selections": [
        {
            "uid": "https://www.functionize.com/page/ai-testing-with-intelligent-test-agent",
            "title": "AI testing with Functionize's intelligent test agent",
            "desc": "Functionize uses artificial intelligence to simplify the whole process of automated testing, from writing new tests to analyzing the test results. AI testing ...",
            "observation": "This item provides information on how to use Functionize's intelligent test agent to test AI agents.",
            "relevant": "Yes"
        },
        {
            "uid": "https://techbeacon.com/app-dev-testing/ai-testing-13-essential-resources-qa-pros",
            "title": "AI in testing: 13 essential resources for Q

In [ ]:
"""
Reflection(text): XXX                   % Gaps in {ai_name}'s knowledge of the Topic
Goal(text): XXX                         % Information needed
Queries[{Q}](text): XXX                        % query for the search engine
Item[{N}](record):                      % a list of item records
> Title(short): XXX                     % 
> Identifier(short): XXX                % Unique ID, can be a URL, filepath, name, ...
> Description(markdow): ```XXX```       % Description provided by the search engine
> Observation(text):```XXX```           % Observation you made about this item
Categories[{C}](record):                % You categorized the items based on topic
> Topic(short): XXX                     % Description of the category
> Items[{N}](short): XXX                % item's identifier, one per line
Sections[{S}](record):                  %
> Topic(short): XXX                     % topic of the section
> Facts[{F}](short): XXX                % list of facts from the
> Paragraphs[{P}](short): XXX           % concepts for each paragraph in the section
Report(record):                         % 
> Title(short): XXX                     % title of the search report
> Introduction(markdown):```XXX```      % Explain your goal and query
> Sections[{S}](record):                % 
> > Subtitle(short): XXX                %   
> > Paragraphs[{P}](markdown):```       %
XXX XXX XXX                             % 
```                                     % 
> Conclusion(markdown):```XXX```        % 
"""